In [1]:
# traing model
# import libary
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, LeakyReLU, Conv2DTranspose, Activation, Reshape
from keras.layers import Conv2D, MaxPooling2D,UpSampling2D,ZeroPadding2D
from keras.layers import BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.backend.tensorflow_backend import set_session

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))
# 設定 Keras 使用的 Session

Using TensorFlow backend.


In [2]:
def getData(i, pack):
    print(i)
    path = '../../celebB/'
    data = np.zeros((pack,10000,64,64,3))
    for i in range(0,pack):
        originImageFile = 'celebA'+str(10000*(i+1))+'.npy'
        data[i] = np.load(path+originImageFile)
        print("A")
    data = np.reshape(data,(-1,64,64,3))
    print(data.shape)
    return data

In [3]:
truncateNormal = keras.initializers.TruncatedNormal(mean=0.0, stddev=0.02, seed=None)
randomNormal = keras.initializers.RandomNormal(mean=0.0, stddev=0.02, seed=None)

In [4]:
def Discriminator():
    
    model = Sequential()
    depth = 64
    alpha=0.2
    # In: 28 x 28 x 1, depth = 1
    # Out: 14 x 14 x 1, depth=64
    input_shape = (64, 64, 3)
    model.add(Conv2D(depth*1, 5, strides=2, kernel_initializer='lecun_normal', input_shape=input_shape, padding='same'))
#    model.add(BatchNormalization(momentum=0.9, epsilon=1e-5))
    model.add(LeakyReLU(alpha))

    model.add(Conv2D(depth*2, 5, strides=2, kernel_initializer='lecun_normal', padding='same'))
#    model.add(BatchNormalization(momentum=0.9, epsilon=1e-5))
    model.add(LeakyReLU(alpha))

    model.add(Conv2D(depth*4, 5, strides=2, kernel_initializer='lecun_normal', padding='same'))
#    model.add(BatchNormalization(momentum=0.9, epsilon=1e-5))
    model.add(LeakyReLU(alpha))

    model.add(Conv2D(depth*8, 5, strides=2, kernel_initializer='lecun_normal', padding='same'))
#    model.add(BatchNormalization(momentum=0.9, epsilon=1e-5))
    model.add(LeakyReLU(alpha))

    # Out: 1-dim probability
    model.add(Flatten())
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    return model

In [5]:
def Generator():
    model = Sequential()
    depth = 512
    dim = 4
    # In: 100
    # Out: dim x dim x depth
    model.add(Dense(dim*dim*depth, kernel_initializer='lecun_normal', input_dim=100))
    model.add(BatchNormalization(momentum=0.9, epsilon=1e-5))
    model.add(Reshape((dim, dim, depth)))

    # In: dim x dim x depth
    # Out: 2*dim x 2*dim x depth/2
    model.add(Conv2DTranspose(int(depth/2), 5, kernel_initializer='lecun_normal', strides=2, padding='same'))
    model.add(BatchNormalization(momentum=0.9, epsilon=1e-5))
    model.add(Activation('relu'))

    model.add(Conv2DTranspose(int(depth/4), 5, kernel_initializer='lecun_normal', strides=2, padding='same'))
    model.add(BatchNormalization(momentum=0.9, epsilon=1e-5))
    model.add(Activation('relu'))

    model.add(Conv2DTranspose(int(depth/8), 5, kernel_initializer='lecun_normal', strides=2, padding='same'))
    model.add(BatchNormalization(momentum=0.9, epsilon=1e-5))
    model.add(Activation('relu'))

    # Out: 28 x 28 x 1 grayscale image [0.0,1.0] per pix
    model.add(Conv2DTranspose(3, 5, strides=2, kernel_initializer='lecun_normal', padding='same'))
    model.add(Activation('tanh'))
    return model

In [ ]:
discriminator = Discriminator()
generator = Generator()
discriminator.summary()
generator.summary()

discriminatorModel = Sequential()
discriminator.trainable = True
discriminatorModel.add(discriminator)
discriminatorModel.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=0.0002),metrics=['accuracy'])

adversarialModel = Sequential()
adversarialModel.add(generator)
discriminator.trainable = False
adversarialModel.add(discriminator)
adversarialModel.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=0.0002),metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 64)        4864      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 128)       204928    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 256)         819456    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 4, 4, 512)         3277312   
__________

In [ ]:
trainSteps = 1000
batch_size = 64

print("training...")
for j in range(trainSteps):
    
    bigdata = getData(j%20, 1)
    bigdata = np.array_split(bigdata,10000/batch_size+1)
    td_loss = 0
    td_acc = 0
    ta_loss = 0
    ta_acc = 0
    for i in range(0,len(bigdata)):
    
        data = bigdata[i]
        
        noise = np.random.normal(-1.0, 1.0, size=[len(data), 100])
        images_fake = generator.predict(noise)
        images_true = data
        x = np.concatenate((images_true, images_fake))
        y = np.ones([len(images_true) + len(images_fake), 1])
        y[len(images_true):, :] = 0
        discriminator.trainable = True
        d_loss = discriminatorModel.train_on_batch(x, y)
        discriminator.trainable = False
        a_loss = adversarialModel.train_on_batch(noise, np.ones([len(data), 1]))
        log = "%5d d:%f %f a:%f %f"%(j+1,d_loss[0],d_loss[1],a_loss[0],a_loss[1])
        td_loss += d_loss[0]
        td_acc += d_loss[1]
        ta_loss += a_loss[0]
        ta_acc += a_loss[1]
        print(log)
    log = "%5d d:%f %f a:%f %f"%(j+1,td_loss/len(bigdata),td_acc/len(bigdata),ta_loss/len(bigdata),ta_acc/len(bigdata))
    filename = "%d_%f_%f_%f_%f"%(j+1,td_loss/len(bigdata),td_acc/len(bigdata),ta_loss/len(bigdata),ta_acc/len(bigdata))
    f = open("records/"+filename,"w")
    f.close()
    print("save image {}".format(log))
    sys.stdout.write('\n')
    images_fake = generator.predict(noise)
    
    width = 6
    new_im = Image.new('RGB', (64*width,64*width))
    for ii in range(width):
        for jj in range(width):
            index=ii*width+jj
            image = (images_fake[index]/2+0.5)*256
            image = image.astype(np.uint8)
            new_im.paste(Image.fromarray(image,"RGB"), (64*ii,64*jj))
    filename = "images/fakeFace%05d.png" % (j+1)
    new_im.save(filename)

training...
0
A
(10000, 64, 64, 3)
    1 d:0.690241 0.500000 a:0.037674 1.000000
    1 d:0.102430 0.898438 a:0.703190 0.000000
    1 d:0.082617 0.929688 a:0.188291 0.906250
    1 d:0.037114 1.000000 a:0.347511 0.078125
    1 d:0.027717 1.000000 a:0.325286 0.093750
    1 d:0.033559 0.984375 a:0.553519 0.000000
    1 d:0.027512 1.000000 a:0.394761 0.015625
    1 d:0.029680 0.992188 a:1.069743 0.000000
    1 d:0.039208 0.976562 a:0.140868 0.953125
    1 d:0.083002 0.960938 a:1.910985 0.000000
    1 d:0.110555 0.906250 a:0.150979 0.921875
    1 d:0.100407 0.968750 a:1.308557 0.000000
    1 d:0.080006 0.906250 a:0.426178 0.000000
    1 d:0.048588 0.984375 a:1.225925 0.000000
    1 d:0.026633 0.992188 a:0.577302 0.000000
    1 d:0.034252 1.000000 a:1.440169 0.000000
    1 d:0.035483 0.992188 a:0.459545 0.000000
    1 d:0.062005 0.992188 a:1.676652 0.000000
    1 d:0.056891 0.984375 a:0.456004 0.000000
    1 d:0.074463 0.976562 a:1.464598 0.000000
    1 d:0.043090 1.000000 a:0.644203 0.000000

    2 d:0.036606 0.984375 a:0.767170 0.000000
    2 d:0.062375 1.000000 a:1.550384 0.000000
    2 d:0.041380 0.984375 a:0.883353 0.000000
    2 d:0.050674 0.992188 a:1.371064 0.000000
    2 d:0.028836 0.984375 a:0.932994 0.000000
    2 d:0.059701 1.000000 a:1.716303 0.000000
    2 d:0.048572 0.992188 a:0.816418 0.000000
    2 d:0.062148 0.992188 a:1.446468 0.000000
    2 d:0.036306 0.992188 a:0.874171 0.000000
    2 d:0.033264 0.992188 a:1.241075 0.000000
    2 d:0.019425 0.992188 a:0.922096 0.000000
    2 d:0.025790 1.000000 a:1.287373 0.000000
    2 d:0.018436 1.000000 a:0.860038 0.000000
    2 d:0.029873 1.000000 a:1.425735 0.000000
    2 d:0.029825 0.992188 a:0.672253 0.000000
    2 d:0.060327 0.992188 a:1.463020 0.000000
    2 d:0.044744 0.976562 a:0.756905 0.000000
    2 d:0.037868 1.000000 a:1.192912 0.000000
    2 d:0.020404 0.992188 a:0.843568 0.000000
    2 d:0.024950 1.000000 a:1.121785 0.000000
    2 d:0.018764 0.992188 a:0.851407 0.000000
    2 d:0.030154 1.000000 a:1.3058

    3 d:0.036267 0.984375 a:1.220402 0.000000
    3 d:0.047294 0.929688 a:0.474038 0.000000
    3 d:0.083962 0.984375 a:1.803365 0.000000
    3 d:0.074436 0.937500 a:0.654753 0.000000
    3 d:0.053518 0.984375 a:0.961646 0.000000
    3 d:0.029533 0.976562 a:0.801222 0.000000
    3 d:0.027692 1.000000 a:1.083743 0.000000
    3 d:0.029829 0.976562 a:0.684203 0.000000
    3 d:0.042301 0.992188 a:1.250732 0.000000
    3 d:0.064701 0.953125 a:0.451318 0.000000
    3 d:0.098529 0.968750 a:1.458277 0.000000
    3 d:0.074166 0.929688 a:0.758101 0.000000
    3 d:0.032139 0.992188 a:0.871761 0.000000
    3 d:0.030323 0.992188 a:0.873971 0.000000
    3 d:0.021443 1.000000 a:1.020563 0.000000
    3 d:0.030403 0.976562 a:0.697732 0.000000
    3 d:0.043518 1.000000 a:1.433458 0.000000
    3 d:0.064968 0.929688 a:0.641290 0.000000
    3 d:0.059691 0.984375 a:1.062985 0.000000
    3 d:0.042405 0.976562 a:0.645361 0.000000
    3 d:0.065341 0.953125 a:1.360028 0.000000
    3 d:0.073760 0.929688 a:0.7353

    4 d:0.068712 0.984375 a:0.949077 0.000000
    4 d:0.065117 0.921875 a:0.445248 0.046875
    4 d:0.046923 0.984375 a:0.615263 0.000000
    4 d:0.045259 0.968750 a:0.380668 0.156250
    4 d:0.041291 0.992188 a:0.707899 0.000000
    4 d:0.043526 0.976562 a:0.350866 0.250000
    4 d:0.054522 0.984375 a:0.803195 0.000000
    4 d:0.069688 0.937500 a:0.277267 0.437500
    4 d:0.081142 0.945312 a:0.865275 0.000000
    4 d:0.066632 0.929688 a:0.388374 0.156250
    4 d:0.052803 0.984375 a:0.668961 0.000000
    4 d:0.036632 0.992188 a:0.457350 0.015625
    4 d:0.055994 0.968750 a:0.775360 0.000000
    4 d:0.054439 0.937500 a:0.370310 0.140625
    4 d:0.067009 0.968750 a:0.892571 0.000000
    4 d:0.067913 0.929688 a:0.367805 0.218750
    4 d:0.077496 0.960938 a:0.910411 0.000000
    4 d:0.085601 0.843750 a:0.447868 0.031250
    4 d:0.071376 0.968750 a:0.749732 0.000000
    4 d:0.049909 0.968750 a:0.462947 0.062500
    4 d:0.058247 0.984375 a:0.938204 0.000000
    4 d:0.055733 0.953125 a:0.3803

    5 d:0.095092 0.875000 a:0.533281 0.000000
    5 d:0.057232 0.984375 a:0.815554 0.000000
    5 d:0.059953 0.914062 a:0.503701 0.000000
    5 d:0.061252 0.976562 a:0.878685 0.015625
    5 d:0.079507 0.882812 a:0.369077 0.250000
    5 d:0.096795 0.914062 a:1.190936 0.000000
    5 d:0.088622 0.851562 a:0.509688 0.015625
    5 d:0.079667 0.953125 a:0.882516 0.000000
    5 d:0.077927 0.898438 a:0.448690 0.062500
    5 d:0.087903 0.937500 a:1.123853 0.000000
    5 d:0.082829 0.875000 a:0.538176 0.000000
    5 d:0.071831 0.953125 a:0.835257 0.000000
    5 d:0.059537 0.929688 a:0.460492 0.093750
    5 d:0.086520 0.937500 a:1.205782 0.000000
    5 d:0.118139 0.812500 a:0.473917 0.078125
    5 d:0.095522 0.898438 a:0.958249 0.000000
    5 d:0.085183 0.843750 a:0.504293 0.015625
    5 d:0.068087 0.960938 a:0.912375 0.000000
    5 d:0.061404 0.898438 a:0.575825 0.015625
    5 d:0.060224 1.000000 a:1.078502 0.000000
    5 d:0.084353 0.867188 a:0.495512 0.062500
    5 d:0.090473 0.953125 a:1.0914

    6 d:0.087077 0.882812 a:0.351217 0.250000
    6 d:0.108594 0.851562 a:1.025910 0.000000
    6 d:0.116085 0.859375 a:0.444437 0.093750
    6 d:0.115153 0.851562 a:0.729159 0.000000
    6 d:0.091402 0.867188 a:0.315825 0.375000
    6 d:0.085075 0.929688 a:0.728229 0.000000
    6 d:0.085935 0.890625 a:0.300630 0.453125
    6 d:0.112986 0.843750 a:0.925246 0.000000
    6 d:0.128564 0.781250 a:0.346818 0.218750
    6 d:0.114139 0.882812 a:0.791258 0.000000
    6 d:0.100308 0.841270 a:0.290442 0.396825
    6 d:0.095013 0.912698 a:0.699004 0.015873
    6 d:0.109023 0.841270 a:0.284205 0.492064
    6 d:0.101290 0.920635 a:0.850159 0.000000
    6 d:0.086175 0.865079 a:0.376371 0.126984
    6 d:0.104764 0.873016 a:0.740002 0.000000
    6 d:0.081432 0.896825 a:0.348429 0.190476
    6 d:0.090794 0.928572 a:1.049976 0.000000
    6 d:0.126252 0.793651 a:0.428622 0.015873
    6 d:0.130545 0.865079 a:0.769559 0.000000
    6 d:0.101782 0.865079 a:0.336290 0.238095
    6 d:0.103344 0.936508 a:1.0234

In [8]:
generator.save("fakeFaceModel.h5")